# Run Agent in Environment with different Algorithms

In [1]:
# Basics
import os
import warnings

# Data 
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import numpy as np

# Logging
import logging
import wandb
from wandb.integration.sb3 import WandbCallback

from gym import make

## Register the Environment 

In [2]:
from gym.envs.registration import register
   
register(
    id="VPPBiddingEnv-TRAIN-v1",
    entry_point='vpp-gym.vpp_gym.envs.vpp_env:VPPBiddingEnv',
    max_episode_steps=1,
    kwargs={'config_path': "vpp_config_4.json",
            'log_level' : "DEBUG", # "DEBUG" , "INFO" or  "WARNING"
            'env_type' :"training",
           }
)

register(
    id="VPPBiddingGoalEnv-TRAIN-v1",
    entry_point='vpp-gym.vpp_gym.envs.vpp_GoalEnv:VPPBiddingGoalEnv',
    max_episode_steps=1,
    kwargs={'config_path': "vpp_config_4.json",
            'log_level' : "DEBUG", # "DEBUG" , "INFO" or  "WARNING"
            'env_type' :"training",
           }
)

register(
    id="VPPBiddingEnv-EVAL-v1",
    entry_point='vpp-gym.vpp_gym.envs.vpp_env:VPPBiddingEnv',
    max_episode_steps=1,
    kwargs={'config_path': "vpp_config_4.json",
            'log_level' : "DEBUG", # "DEBUG" , "INFO" or  "WARNING"
            'env_type' :"eval",
           }
)

register(
    id="VPPBiddingEnv-TEST-v1",
    entry_point='vpp-gym.vpp_gym.envs.vpp_env:VPPBiddingEnv',
    max_episode_steps=1,
    kwargs={'config_path': "vpp_config_4.json",
            'log_level' : "INFO", # "DEBUG" , "INFO" or  "WARNING"
            'env_type' :"test",
           }
)

In [3]:
from stable_baselines3.common.env_checker import check_env
# It will check your custom environment and output additional warnings if needed
env_to_check = make('VPPBiddingEnv-TEST-v1')
check_env(env_to_check)


log level = info
log level = warning
Bid Submission time (D-1) = 2020-07-02 05:00:00+00:00
Gate Closure time (D-1) = 2020-07-02 06:00:00+00:00
Historic Data Window: from 2020-07-01 05:00:00+00:00 to 2020-07-02 04:45:00+00:00 
Forecast Data Window: from 2020-07-02 22:00:00+00:00 to 2020-07-03 21:45:00+00:00 
Current Slot Time: (D) = 2020-07-02 22:00:00+00:00
agents_bid_size = 101
agents_bid_price = 3467.9412
settlement_price_DE : 16.67
self.delivery_results['slots_won'] = 
slot won: 	0
slot won: 	None
slot won: 	None
slot won: 	None
slot won: 	None
slot won: 	None
     agents bid_size = 
size: 	101
size: 	126
size: 	127
size: 	69
size: 	39
size: 	68
self.delivery_results['slot_settlement_prices_DE'] = 
price: 	16.67
price: 	0.0
price: 	0.0
price: 	0.0
price: 	0.0
price: 	0.0
Current Slot Time: (D) = 2020-07-03 02:00:00+00:00
agents_bid_size = 126
agents_bid_price = 3506.7686
settlement_price_DE : 19.06
self.delivery_results['slots_won'] = 
slot won: 	0
slot won: 	0
slot won: 	None
slot 

# Stable Baselines

## TD3 

### Train

In [ ]:
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 16710 # 557
}

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
#action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=0.34709686 * np.ones(n_actions))

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3"], 
    job_type="training"
)

model = TD3(config['policy'],
            env,
            verbose=0,
            tensorboard_log=f"runs/ddpg",
            gamma=0.99,
            batch_size=200, 
            buffer_size=1000000,
            learning_rate=0.2456,
            tau=0.001,
            action_noise=action_noise,
            policy_kwargs = {'net_arch': [64, 64]}
           )


model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=0))
wandb.finish()


### Eval

In [ ]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3"], 
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

### Tuning TD3

In [ ]:
import optuna
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback


def sample_td3_params(trial: optuna.Trial):
    """
    Sampler for TD3 hyperparams.
    :param trial:
    :return:
    """
    gamma = trial.suggest_categorical("gamma", [0.9, 0.95, 0.98, 0.99, 0.995, 0.999, 0.9999])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 100, 128, 200])
    buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(1e5), int(1e6)])
    # Polyak coeff
    tau = trial.suggest_categorical("tau", [0.001, 0.005, 0.01, 0.02, 0.05, 0.08])

    #train_freq = trial.suggest_categorical("train_freq", [1, 4, 8, 16, 32, 64, 128, 256, 512])
    #gradient_steps = train_freq
    #learning_starts = trial.suggest_int('learning_starts', 0, 200, 10)  #default: 100


    noise_type = trial.suggest_categorical("noise_type", ["ornstein-uhlenbeck", "normal", None])
    noise_std = trial.suggest_uniform("noise_std", 0, 1)

    # NOTE: Add "verybig" to net_arch when tuning HER
    net_arch = trial.suggest_categorical("net_arch", ["small", "medium", "big"])
    # activation_fn = trial.suggest_categorical('activation_fn', [nn.Tanh, nn.ReLU, nn.ELU, nn.LeakyReLU])

    net_arch = {
        "small": [64, 64],
        "medium": [256, 256],
        "big": [400, 300],
        # Uncomment for tuning HER
        # "verybig": [256, 256, 256],
    }[net_arch]

    hyperparams = {
        "gamma": gamma,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "buffer_size": buffer_size,
        #"train_freq": train_freq,
        #"gradient_steps": gradient_steps,
        "policy_kwargs": dict(net_arch=net_arch),
        "tau": tau,
    }
    
    n_actions = 12      
    if noise_type == "normal":
        hyperparams["action_noise"] = NormalActionNoise(
            mean=np.zeros(n_actions), sigma=noise_std * np.ones(n_actions)
        )
    elif noise_type == "ornstein-uhlenbeck":
        hyperparams["action_noise"] = OrnsteinUhlenbeckActionNoise(
            mean=np.zeros(n_actions), sigma=noise_std * np.ones(n_actions)
        )

    return hyperparams




def optimize_agent_td3(trial):
    """ Train the model and optimize
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
   
    
    model_params = sample_td3_params(trial)
    
    # init tracking experiment.
    # hyper-parameters, trial id are stored.
    config = dict(trial.params)
    config["trial.number"] = trial.number
    wandb.init(
        project="RL-optuna",
        entity="jlu237", 
        sync_tensorboard=True,
        config=config,
        tags=["TD3"],
        reinit=True
    )
    
    env = make('VPPBiddingEnv-TRAIN-v1')
    env = Monitor(env) 
    env = RecordEpisodeStatistics(env) # record stats such as returns
    
    
    model = TD3('MultiInputPolicy', env, verbose=0, tensorboard_log=f"runs/td3", seed = 1, **model_params)

    model.learn(total_timesteps=2785,
                log_interval=1,
                callback=WandbCallback(
                    gradient_save_freq=1,
                    verbose=0))

    wandb.finish()
    
study = optuna.create_study()
try:
    study.optimize(optimize_agent_td3, n_trials=20)
except KeyboardInterrupt:
    print('Interrupted by keyboard.')

## A2C 

### Train

In [4]:
from stable_baselines3 import A2C
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 2785 #557
}

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["A2C"], 
    job_type="training"
)

model = A2C(config['policy'], env, verbose=1)


model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


wandb: Currently logged in as: jlu237. Use `wandb login --relogin` to force relogin


Using cpu device
Wrapping the env in a DummyVecEnv.
-----------------------------------
| rollout/           |            |
|    ep_len_mean     | 1          |
|    ep_rew_mean     | -16297.659 |
| time/              |            |
|    fps             | 0          |
|    iterations      | 1          |
|    time_elapsed    | 5          |
|    total_timesteps | 5          |
-----------------------------------
--------------------------------------
| rollout/              |            |
|    ep_len_mean        | 1          |
|    ep_rew_mean        | -31470.613 |
| time/                 |            |
|    fps                | 1          |
|    iterations         | 2          |
|    time_elapsed       | 7          |
|    total_timesteps    | 10         |
| train/                |            |
|    entropy_loss       | -17        |
|    explained_variance | 1.37e-06   |
|    learning_rate      | 0.0007     |
|    n_updates          | 1          |
|    policy_loss        | -2.92e+05  |
|  

wandb: Network error (ConnectionError), entering retry loop.


-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1         |
|    ep_rew_mean        | -5478.656 |
| time/                 |           |
|    fps                | 2         |
|    iterations         | 221       |
|    time_elapsed       | 506       |
|    total_timesteps    | 1105      |
| train/                |           |
|    entropy_loss       | -17       |
|    explained_variance | nan       |
|    learning_rate      | 0.0007    |
|    n_updates          | 220       |
|    policy_loss        | -1.04e+04 |
|    std                | 0.998     |
|    value_loss         | 3.39e+05  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1         |
|    ep_rew_mean        | -9288.638 |
| time/                 |           |
|    fps                | 2         |
|    iterations         | 222       |
|    time_elapsed       | 521       |
|    total_t

wandb: ERROR Control-C detected -- Run data was not synced


### Eval 

In [6]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3"], 
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

wandb: WARNING Step cannot be set when using syncing with tensorboard. Please log your step values as a metric such as 'global_step'


Episode: 0 Info: {'bid_submission_time': '2020-08-02 05:00:00+00:00', 'step_reward': -600, 'total_reward': -600.0, 'total_profit': 0.0, 'TimeLimit.truncated': False, 'episode': {'r': -600.0, 'l': 1, 't': 9.174277}}
Episode: 1 Info: {'bid_submission_time': '2020-08-03 05:00:00+00:00', 'step_reward': -6461.3, 'total_reward': -7061.3, 'total_profit': -3030.65, 'TimeLimit.truncated': False, 'episode': {'r': -6461.303, 'l': 1, 't': 10.166581}}
Episode: 2 Info: {'bid_submission_time': '2020-08-04 05:00:00+00:00', 'step_reward': -600, 'total_reward': -7661.3, 'total_profit': -3030.65, 'TimeLimit.truncated': False, 'episode': {'r': -600.0, 'l': 1, 't': 10.654331}}
Episode: 3 Info: {'bid_submission_time': '2020-08-05 05:00:00+00:00', 'step_reward': -600, 'total_reward': -8261.3, 'total_profit': -3030.65, 'TimeLimit.truncated': False, 'episode': {'r': -600.0, 'l': 1, 't': 11.123091}}
Episode: 4 Info: {'bid_submission_time': '2020-08-06 05:00:00+00:00', 'step_reward': -600, 'total_reward': -8861.

episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
episode_reward,▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇█▇▇█▇▇▇▇▇▇▄▇▇▇▇▇▇▇▁▇▇▇▇▇
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
step_reward,▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇█▇▇█▇▇▇▇▇▇▄▇▇▇▇▇▇▇▁▇▇▇▇▇
total_profit,███▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████▇▇▇▇▇▇▇▇▆▅▂▂▂▁
total_reward,████████▇▇▇▇▇▇▇▇▇████████▇▇▇▇▇▇▇▇▇▅▅▂▂▂▁
episode,139
episode_reward,-600
global_step,139
mean_run_reward,-6803.7205
step_reward,-600


## PPO

### Train

In [7]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 2785 #557
}

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["PPO"], 
    job_type="training"
)

model = PPO(config['policy'], env, verbose=1)


model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


Using cpu device
Wrapping the env in a DummyVecEnv.
-----------------------------------
| rollout/           |            |
|    ep_len_mean     | 1          |
|    ep_rew_mean     | -61100.406 |
| time/              |            |
|    fps             | 3          |
|    iterations      | 1          |
|    time_elapsed    | 629        |
|    total_timesteps | 2048       |
-----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -10448.87   |
| time/                   |             |
|    fps                  | 3           |
|    iterations           | 2           |
|    time_elapsed         | 1222        |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007294045 |
|    clip_fraction        | 0.0373      |
|    clip_range           | 0.2         |
|    entropy_loss         | -17         |


global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
step_reward,▇█▇███████▇▆▇▆▅▁▃▆█▆███▇▄▆██████▇█████▆▆
total_profit,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
total_reward,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
global_step,4095
step_reward,-600
total_profit,-70037765.38624
total_reward,-124256986.90818


### Eval 

In [9]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["PPO"], 
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

Episode: 0 Info: {'bid_submission_time': '2020-08-02 05:00:00+00:00', 'step_reward': -45968.75, 'total_reward': -45968.75, 'total_profit': -22784.37, 'TimeLimit.truncated': False, 'episode': {'r': -45968.746, 'l': 1, 't': 6.601461}}
Episode: 1 Info: {'bid_submission_time': '2020-08-03 05:00:00+00:00', 'step_reward': -4673.54, 'total_reward': -50642.29, 'total_profit': -24921.14, 'TimeLimit.truncated': False, 'episode': {'r': -4673.542, 'l': 1, 't': 7.509735}}
Episode: 2 Info: {'bid_submission_time': '2020-08-04 05:00:00+00:00', 'step_reward': -23094.14, 'total_reward': -73736.43, 'total_profit': -36268.21, 'TimeLimit.truncated': False, 'episode': {'r': -23094.139, 'l': 1, 't': 8.086394}}
Episode: 3 Info: {'bid_submission_time': '2020-08-05 05:00:00+00:00', 'step_reward': -600, 'total_reward': -74336.43, 'total_profit': -36268.21, 'TimeLimit.truncated': False, 'episode': {'r': -600.0, 'l': 1, 't': 8.686861}}
Episode: 4 Info: {'bid_submission_time': '2020-08-06 05:00:00+00:00', 'step_rew

episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
episode_reward,██▅▇█▆█▆████▇████████▇████▇▃▃▃█▅▄███▁███
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
step_reward,██▅▇█▆█▆████▇████████▇████▇▃▃▃█▅▄███▁███
total_profit,████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▅▅▄▄▃▂▂▂▁▁▁▁
total_reward,████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▅▅▄▄▃▂▂▂▁▁▁▁
episode,139
episode_reward,2570.0
global_step,139
mean_run_reward,-23107.809
step_reward,2570.0


## SAC

### Train

In [10]:
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 2785 #557
}

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["SAC"], 
    job_type="training"
)

model = SAC(config['policy'], env, verbose=1)


model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


Using cpu device
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -600.0   |
| time/              |          |
|    episodes        | 1        |
|    fps             | 4        |
|    time_elapsed    | 0        |
|    total_timesteps | 1        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -600.0   |
| time/              |          |
|    episodes        | 2        |
|    fps             | 2        |
|    time_elapsed    | 0        |
|    total_timesteps | 2        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -600.0   |
| time/              |          |
|    episodes        | 3        |
|    fps             | 2        |
|    time_elapsed    | 1      

global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
step_reward,██████▁▄██▇█████████████████████████████
total_profit,█████▆▅▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_reward,█████▆▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,2784
step_reward,-124.8
total_profit,-11732007.78982
total_reward,-20146995.41033


### Eval 

In [11]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["SAC"], 
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

Episode: 0 Info: {'bid_submission_time': '2020-08-02 05:00:00+00:00', 'step_reward': -153.82, 'total_reward': -153.82, 'total_profit': 23.09, 'TimeLimit.truncated': False, 'episode': {'r': -153.82, 'l': 1, 't': 7.757353}}
Episode: 1 Info: {'bid_submission_time': '2020-08-03 05:00:00+00:00', 'step_reward': 196.9, 'total_reward': 43.08, 'total_profit': 83.86, 'TimeLimit.truncated': False, 'episode': {'r': 196.90155, 'l': 1, 't': 8.288107}}
Episode: 2 Info: {'bid_submission_time': '2020-08-04 05:00:00+00:00', 'step_reward': -248.67, 'total_reward': -205.59, 'total_profit': 109.52, 'TimeLimit.truncated': False, 'episode': {'r': -248.67381, 'l': 1, 't': 8.543253}}
Episode: 3 Info: {'bid_submission_time': '2020-08-05 05:00:00+00:00', 'step_reward': 91.84, 'total_reward': -113.75, 'total_profit': 167.2, 'TimeLimit.truncated': False, 'episode': {'r': 91.8406, 'l': 1, 't': 8.833054}}
Episode: 4 Info: {'bid_submission_time': '2020-08-06 05:00:00+00:00', 'step_reward': 86.37, 'total_reward': -27.

episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
episode_reward,▁▁▁▂▂▁▂▂▁▂▁▂▂▂▂▁▂▂▂▂▂▂▁▂▂▂▂▁▂▁▂▂▂▂█▃▂▂▂▂
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
step_reward,▁▁▁▂▂▁▂▂▁▂▁▂▂▂▂▁▂▂▂▂▂▂▁▂▂▂▂▁▂▁▂▂▂▂█▃▂▂▂▂
total_profit,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▅▅▆▇██
total_reward,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▅▅▆▇██
episode,139
episode_reward,546.76
global_step,139
mean_run_reward,523.267
step_reward,546.76


## HER 

- HER is an algorithm that works with off-policy methods (SAC,TQC, TD3 and DDPG)
- HER is no longer a separate algorithm but a replay buffer class HerReplayBuffer that must be passed to an off-policy algorithm when using MultiInputPolicy (to have Dict observation support).
- HER requires the environment to inherits from gym.GoalEnv
- For performance reasons, the maximum number of steps per episodes must be specified. In most cases, it will be inferred if you specify max_episode_steps when registering the environment or if you use a gym.wrappers.TimeLimit (and env.spec is not None). Otherwise, you can directly pass max_episode_length to the model constructor

### Train

In [ ]:
from stable_baselines3 import HerReplayBuffer, DDPG, SAC, TD3
from stable_baselines3.her.goal_selection_strategy import GoalSelectionStrategy

model_class = TD3  # works also with SAC, DDPG and TD3

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns

config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 2785 #557
}

# Available strategies (cf paper): future, final, episode
goal_selection_strategy = 'future' # equivalent to GoalSelectionStrategy.FUTURE

# If True the HER transitions will get sampled online
online_sampling = True

# Time limit for the episodes
max_episode_length = 1

# Initialize the model
model = model_class(
    "MultiInputPolicy",
    env,
    replay_buffer_class=HerReplayBuffer,
    # Parameters for HER
    replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy=goal_selection_strategy,
        online_sampling=online_sampling,
        max_episode_length=max_episode_length,
    ),
    verbose=1,
)

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["normalized_obs","new_action_high_price", "new_action_high_size", "delivery_simulation_4.0", "TD3","training", "wind_config", "single vpp obs", "vpp config", "13MW config", "4kprice", "updated_reward" , "delivery_against_FCR", "pred_market_prices"], 
    job_type="training"
)


# Train the model
model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


### Eval 

In [ ]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3", "normalized_obs", "wind_config","eval", "single vpp obs", "vpp config", "price plots","updated_reward" , "delivery_against_FCR", "pred_market_prices"],
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

## Contrib packages: ARS

### Train

In [15]:
from sb3_contrib import ARS
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 2785 #557
}

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["ARS"], 
    job_type="training"
)

model = ARS(config['policy'], env, verbose=1)


model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


ValueError: Policy MultiInputPolicy unknown

### Eval 

In [ ]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3", "normalized_obs", "wind_config","eval", "single vpp obs", "vpp config", "price plots","updated_reward" , "delivery_against_FCR", "pred_market_prices"],
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

## Contrib packages: RecurrentPPO 

### Train

In [ ]:
from sb3-contrib import RecurrentPPO
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 2785 #557
}

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["RecurrentPPO"], 
    job_type="training"
)

model = RecurrentPPO(config['policy'], env, verbose=1)


model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


### Eval 

In [ ]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3", "normalized_obs", "wind_config","eval", "single vpp obs", "vpp config", "price plots","updated_reward" , "delivery_against_FCR", "pred_market_prices"],
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

## Contrib packages: TQC

### Train

In [ ]:
from sb3-contrib import TQC
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 2785 #557
}

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TQC"], 
    job_type="training"
)

model = TQC(config['policy'], env, verbose=1)


model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


### Eval 

In [ ]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3", "normalized_obs", "wind_config","eval", "single vpp obs", "vpp config", "price plots","updated_reward" , "delivery_against_FCR", "pred_market_prices"],
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

## Contrib packages: TRPO

### Train

In [ ]:
from sb3-contrib import TRPO
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 2785 #557
}

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TRPO"], 
    job_type="training"
)

model = TRPO(config['policy'], env, verbose=1)


model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


### Eval 

In [ ]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3", "normalized_obs", "wind_config","eval", "single vpp obs", "vpp config", "price plots","updated_reward" , "delivery_against_FCR", "pred_market_prices"],
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

## DDPG: Deep Deterministic Policy Gradient (DDPG) 

### Train

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
from gym.wrappers import RecordEpisodeStatistics
from wandb.integration.sb3 import WandbCallback

env = make('VPPBiddingEnv-TRAIN-v1')
env = Monitor(env) 
env = RecordEpisodeStatistics(env) # record stats such as returns


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 557
}

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma= 0.1 * np.ones(n_actions))

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Training",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["DDPG"], 
    job_type="training"
)

model = DDPG(config['policy'], env, action_noise=action_noise, verbose=1, tensorboard_log=f"runs/ddpg")

model.learn(total_timesteps=config['total_timesteps'],
            log_interval=1,
            callback=WandbCallback(
                gradient_save_freq=1,
                verbose=2))
wandb.finish()


### Eval 

In [ ]:
from stable_baselines3.common.monitor import Monitor

eval_env = make('VPPBiddingEnv-EVAL-v1')
eval_env = RecordEpisodeStatistics(eval_env) # record stats such as returns

wandb.init(
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="RL-VPP-Evaluation",
    #monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
    entity="jlu237", 
    tags=["TD3", "normalized_obs", "wind_config","eval", "single vpp obs", "vpp config", "price plots","updated_reward" , "delivery_against_FCR", "pred_market_prices"],
    job_type="eval"
)


tbl = wandb.Table(columns=["episode", "bid_submission_time"])

episodes = 140

for i_episode in range(episodes):
    observation = eval_env.reset()
    for t in range(1):
        eval_env.render(mode="human")
        logging.debug("observation : " + str(observation))
        action, _states = model.predict(observation)
        observation, reward, done, info = eval_env.step(action)
        if done:
            print('Episode: {} Info: {}'.format(i_episode, info))
            tbl.add_data(i_episode, info["bid_submission_time"])
            wandb.log({"episode_reward": reward,
                       "episode": i_episode
                      })
            
            break
wandb.log({"bid_submission_time" : tbl})

eval_env.close()
mean_run_reward = info["total_reward"] / episodes

wandb.run.summary["mean_run_reward"] = mean_run_reward
print("Mean Run Reward: " + str(mean_run_reward))
wandb.finish()

## Tuning DDPG

### Parameters

- policy = "MlpPolicy" , "CnnPolicy" , "MultiInputPolicy"
- **learning_rate** = staic or range(1,0)
- buffer_size (int) – size of the replay buffer
- **learning_starts (int)** – how many steps of the model to collect transitions for before learning starts
    -  For a fixed number of steps at the beginning (set with the start_steps keyword argument), the agent takes actions which are sampled from a uniform random distribution over valid actions. After that, it returns to normal DDPG exploration.
- batch_size (int) – Minibatch size for each gradient update
- **tau (float)** – the soft update coefficient (“Polyak update”, between 0 and 1)
- gamma (float) – the discount factor
- train_freq (Union[int, Tuple[int, str]]) – Update the model every train_freq steps. Alternatively pass a tuple of frequency and unit like (5, "step") or (2, "episode").
- gradient_steps (int) – How many gradient steps to do after each rollout (see train_freq) Set to -1 means to do as many gradient steps as steps done in the environment during the rollout.
- action_noise (Optional[ActionNoise]) – the action noise type (None by default), this can help for hard exploration problem. Cf common.noise for the different action noise type.
    -  uncorrelated, mean-zero Gaussian noise works perfectly well. 
    -  To facilitate getting higher-quality training data, you may reduce the scale of the noise over the course of training. (We do not do this in our implementation, and keep noise scale fixed throughout.)


- replay_buffer_class (Optional[ReplayBuffer]) – Replay buffer class to use (for instance HerReplayBuffer). If None, it will be automatically selected.
- optimize_memory_usage (bool) – Enable a memory efficient variant of the replay buffer at a cost of more complexity. See https://github.com/DLR-RM/stable-baselines3/issues/37#issuecomment-637501195
- create_eval_env (bool) – Whether to create a second environment that will be used for evaluating the agent periodically. (Only available when passing string for the environment)

- seed (Optional[int]) – Seed for the pseudo random generators
- _init_setup_model (bool) – Whether or not to build the network at the creation of the instance





stable_baselines3.ddpg.MlpPolicy Parameters
- lr_schedule (Callable[[float], float]) – Learning rate schedule (could be constant)
- n_critics (int) – Number of critic networks to create.

stable_baselines3.ddpg.MlpPolicy.set_training_mode()
- mode (bool) – if true, set to training mode, else set to evaluation mode

stable_baselines3.ddpg.CnnPolicy

stable_baselines3.ddpg.MultiInputPolicy


In [ ]:
# hide all deprecation warnings from tensorflow
#import tensorflow as tf
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import optuna

#from stable_baselines import PPO2
from stable_baselines3 import DDPG
from stable_baselines3 import HerReplayBuffer
from gym.wrappers import RecordEpisodeStatistics
from stable_baselines3.common.noise import NormalActionNoise
#from stable_baselines.common.evaluation import evaluate_policy
#from stable_baselines.common.cmd_util import make_vec_env

# https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/5_custom_gym_env.ipynb
#from custom_env import GoLeftEnv

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
normal_action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))


def optimize_ddpg(trial):
    """ Learning hyperparamters we want to optimise"""
    
    replay_buffer_class = trial.suggest_categorical("replay_buffer_class", ["HER", "None"])
    replay_buffer_class = {"HER": HerReplayBuffer, "None": None}[replay_buffer_class]
    
    action_noise = trial.suggest_categorical("action_noise", ["action_noise", "None"])
    action_noise = {"action_noise": normal_action_noise, "None": None}[action_noise]
    
    params =  {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 1.0), #default: 0.001
        'learning_starts': int(trial.suggest_int('learning_starts', 0, 200, 10)),  #default: 100
        'batch_size': int(trial.suggest_int('batch_size', 0, 200,10)),  #default: 100
        'tau': trial.suggest_loguniform('tau', 0.001, 1.0), #default: 0.005
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999), # default: gamma=0.99
        'replay_buffer_class' : replay_buffer_class,
        'action_noise' : action_noise
    }
    
    return params
        



def optimize_agent(trial):
    """ Train the model and optimize
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    
    model_params = optimize_ddpg(trial)
    
    # init tracking experiment.
    # hyper-parameters, trial id are stored.
    config = dict(trial.params)
    config["trial.number"] = trial.number
    wandb.init(
        project="RL-optuna",
        entity="jlu237", 
        sync_tensorboard=True,
        config=config,
        reinit=True
    )
    
    env = make('VPPBiddingEnv-TRAIN-v1')
    env = Monitor(env) 
    env = RecordEpisodeStatistics(env) # record stats such as returns


    model = DDPG('MultiInputPolicy', env, verbose=0, tensorboard_log=f"runs/ddpg", seed = 1, **model_params)
    model.learn(total_timesteps=557, log_interval=1)
    
    wandb.finish()
    
study = optuna.create_study()
try:
    study.optimize(optimize_agent, n_trials=20)
except KeyboardInterrupt:
    print('Interrupted by keyboard.')

In [ ]:
env = make('VPPBiddingEnv-TRAIN-v1')
env.observation_space.spaces["observation"]

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
model.get_parameters()["critic.optimizer"]["param_groups"]

In [ ]:
model.get_parameters()["actor.optimizer"]["param_groups"]

In [ ]:
# !apt-get install swig cmake ffmpeg freeglut3-dev xvfb

In [ ]:
# Alternative from araffin for optuna from: https://github.com/optuna/optuna-examples/blob/52ed3aff3e3e936be3873b5acc6ee3ccdadea914/rl/sb3_simple.py#L60

""" Optuna example that optimizes the hyperparameters of
a reinforcement learning agent using A2C implementation from Stable-Baselines3
on a OpenAI Gym environment.

This is a simplified version of what can be found in https://github.com/DLR-RM/rl-baselines3-zoo.

You can run this example as follows:
    $ python sb3_simple.py

"""
from typing import Any
from typing import Dict

import gym
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(2e4)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3

ENV_ID = "CartPole-v1"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "env": ENV_ID,
}


def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for A2C hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    gae_lambda = 1.0 - trial.suggest_float("gae_lambda", 0.001, 0.2, log=True)
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    # Display true values
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("gae_lambda_", gae_lambda)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]} if net_arch == "tiny" else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "gae_lambda": gae_lambda,
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
            "ortho_init": ortho_init,
        },
    }


class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:

    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters
    kwargs.update(sample_a2c_params(trial))
    # Create the RL model
    model = A2C(**kwargs)
    # Create env used for evaluation
    eval_env = gym.make(ENV_ID)
    # Create the callback that will periodically evaluate
    # and report the performance
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))

In [ ]:
# code from https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/utils/hyperparams_opt.py#L340

def sample_ddpg_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for DDPG hyperparams.
    :param trial:
    :return:
    """
    gamma = trial.suggest_categorical("gamma", [0.9, 0.95, 0.98, 0.99, 0.995, 0.999, 0.9999])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 100, 128, 256, 512, 1024, 2048])
    buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(1e5), int(1e6)])
    # Polyak coeff
    tau = trial.suggest_categorical("tau", [0.001, 0.005, 0.01, 0.02, 0.05, 0.08])

    train_freq = trial.suggest_categorical("train_freq", [1, 4, 8, 16, 32, 64, 128, 256, 512])
    gradient_steps = train_freq

    noise_type = trial.suggest_categorical("noise_type", ["ornstein-uhlenbeck", "normal", None])
    noise_std = trial.suggest_uniform("noise_std", 0, 1)

    # NOTE: Add "verybig" to net_arch when tuning HER (see TD3)
    net_arch = trial.suggest_categorical("net_arch", ["small", "medium", "big"])
    # activation_fn = trial.suggest_categorical('activation_fn', [nn.Tanh, nn.ReLU, nn.ELU, nn.LeakyReLU])

    net_arch = {
        "small": [64, 64],
        "medium": [256, 256],
        "big": [400, 300],
    }[net_arch]

    hyperparams = {
        "gamma": gamma,
        "tau": tau,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "buffer_size": buffer_size,
        "train_freq": train_freq,
        "gradient_steps": gradient_steps,
        "policy_kwargs": dict(net_arch=net_arch),
    }

    if noise_type == "normal":
        hyperparams["action_noise"] = NormalActionNoise(
            mean=np.zeros(trial.n_actions), sigma=noise_std * np.ones(trial.n_actions)
        )
    elif noise_type == "ornstein-uhlenbeck":
        hyperparams["action_noise"] = OrnsteinUhlenbeckActionNoise(
            mean=np.zeros(trial.n_actions), sigma=noise_std * np.ones(trial.n_actions)
        )

    if trial.using_her_replay_buffer:
        hyperparams = sample_her_params(trial, hyperparams)

    return hyperparams


In [ ]:
!git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo

In [ ]:
#!cd rl-baselines3-zoo/

In [ ]:
!pip install -r rl-baselines3-zoo/requirements.txt

In [ ]:
!python rl-baselines3-zoo/train.py --algo ddpg --env VPPBiddingEnv-TRAIN-v1 -n 697 -optimize --n-trials 5 --n-jobs -1 \
  --sampler tpe --pruner median

In [ ]:
!python rl-baselines3-zoo/scripts/parse_study.py -i path/to/study.pkl --print-n-best-trials 10 --save-n-best-hyperparameters 10


### PPO - Proximal Policy Optimization algorithm 

#### Train the agent

#### Evaluate Agent

## A2C - synchronous, deterministic variant of Asynchronous Advantage Actor Critic (A3C)

#### Training

#### Eval

## Other Algorithm 

In [ ]:
# todo

## DQN -- needs Discrete Action Space. 

# Testing

#### Run Episodes

### Check the Environment

# Keras 

## 2. Create a Deep Learning Model with Keras

## 3. Build Agent with Keras-RL


## 4. Reloading Agent from Memory


# Archive


### 2. create test list , check if date is in test list, if yes, skip day 

1. data set start date = 01.07.2020
2. training start date = 02.07.2020 
3. first slot lower boundary = 02.07.2020 22.00
4. make test set 
    - take time_features_df
    - substract 2 hours from each timestamp = start of slot 
    - iterate over df and get date every 5 days, add to list = test list. 
5. in training mode -> skip dates in list. 
6. in test mode -> take only dates from test list. 
7. unterschiedliche testsets erstellen? izzy meinte, zusammenhängende woche wäre gut (seasonality)

Vorgehen:
Ab der ersten Vollen Woche: Woche nehmen und Testset-Liste hinzufügen, 
dann skip 5 wochen , dann 1 woche test woche 


### scaler for observations


In [ ]:
# scaler for observations

scaler = MinMaxScaler(feature_range=(-1,1))

a_raw = asset_data_historic
print("a_raw")

print(a_raw)

scaler.fit(np.array(a_raw).reshape(-1, 1))

b_transformed = scaler.transform((a_raw.reshape(-1, 1)))
print("b_transformed")

print(b_transformed)

# convert from array to list
c_list = [x for xs in list(b_transformed) for x in xs]
print("c_list")

print(c_list)

# transform back to 

d_transformed_back = (scaler.inverse_transform(np.array(c_list).reshape(-1, 1)))
print("d_transformed_back")
print(d_transformed_back)

print("e_array")
e_array = d_transformed_back.flatten()
print(e_array)


print("f_list")
f_list = [x for xs in list(d_transformed_back) for x in xs]

print(f_list)